# Reaction-network: Enumerators (Demo Notebook)

### <u> Author:</u> Matthew McDermott (Lawerence Berkeley National Laboratory)
Last Updated: 10/20/22

This notebook is a lesson on how to use the reaction enumerator classes in the _reaction-network_ package. These enumerators supply all reaction data to be used in reaction network construction (or other analyses).

If you use this code in your work, please consider citing the following paper:

    McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

### Imports

In [1]:
import logging
from pprint import pprint

from mp_api.client import MPRester
from pymatgen.core.composition import Composition, Element

from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from rxn_network.costs.softplus import Softplus
from rxn_network.entries.entry_set import GibbsEntrySet
from rxn_network.reactions.reaction_set import ReactionSet

%load_ext autoreload
%autoreload 2

/Users/mcdermott/opt/miniconda3/envs/rn_dev2/lib/python3.10/site-packages/pymatgen/analysis/phase_diagram.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Downloading and modifying entries

First, we acquire entries for phases in the Y-Mn-O chemical system from the Materials Project (MP), a computed materials database containing calculations for over 130,000 materials.

In [2]:
with MPRester() as mpr:  # insert your Materials Project API key here if it's not stored as an environment variable
    entries = mpr.get_entries_in_chemsys("Y-Mn-O")

/Users/mcdermott/opt/miniconda3/envs/rn_dev2/lib/python3.10/site-packages/mp_api/client/client.py:144: builtins.UserWarning: Problem loading MPContribs client: [<twisted.python.failure.Failure OpenSSL.SSL.Error: [('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('SSL routines', '', 'certificate verify failed')]>]


Retrieving ThermoDoc documents:   0%|          | 0/136 [00:00<?, ?it/s]

The `GibbsEntrySet` class allows us to automatically convered `ComputedStructureEntry` objects downloaded from the MP database into `GibbsComputedEntry` objects, where DFT-calculated energies have been converted to an AI-estimated equivalent values of the Gibbs free energies of formation, $\Delta G_f$ for all entries at the specified temperature.

For more information, check out the citation in the documentation for `GibbsComputedEntry`.

In [3]:
temp = 900  # units: Kelvin
gibbs_entries = GibbsEntrySet.from_computed_entries(entries, temp)

We can print the entries by calling `.entries` or `.entries_list`:

In [4]:
gibbs_entries.entries_list  # all entries in the Y-Mn-O system (in alphabetical order)

[GibbsComputedEntry | mp-12957 | O8 (O2)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-1238773 | Mn1 O1 (MnO)
 Gibbs Energy (900 K) = -2.6157,
 GibbsComputedEntry | mp-1238899 | Mn1 O1 (MnO)
 Gibbs Energy (900 K) = -2.9909,
 GibbsComputedEntry | mp-25223 | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = -3.7553,
 GibbsComputedEntry | mp-25424 | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = -3.9034,
 GibbsComputedEntry | mp-1272141 | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = -3.9507,
 GibbsComputedEntry | mp-796077 | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = 0.6043,
 GibbsComputedEntry | mp-1221542 | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = -2.4906,
 GibbsComputedEntry | mp-19006 | Mn2 O2 (MnO)
 Gibbs Energy (900 K) = -6.1654,
 GibbsComputedEntry | mp-510732 | Mn2 O4 (MnO2)
 Gibbs Energy (900 K) = -6.6081,
 GibbsComputedEntry | mp-755671 | Mn2 O4 (MnO2)
 Gibbs Energy (900 K) = -7.6123,
 GibbsComputedEntry | mp-1221574 | Mn2 O4 (MnO2)
 Gibbs Energy (900 K) = -0.4639,
 GibbsComputedEntry | mp-1086672 | Mn2

The `GibbsEntrySet` class has many helpful functions, such as the following `filter_by_stability()` function, which automatically removes entries which are a specified energy per atom above the convex hull of stability:

In [5]:
filtered_entries = gibbs_entries.filter_by_stability(0.025)

You should now see a much shorter list of entries within the Y-Mn-O system (< 25 meV/atom below the hull)

In [6]:
filtered_entries.entries_list

[GibbsComputedEntry | mp-12957 | O8 (O2)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-1272141 | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = -3.9507,
 GibbsComputedEntry | mp-999539 | Mn4 O4 (MnO)
 Gibbs Energy (900 K) = -12.5274,
 GibbsComputedEntry | mp-18922 | Mn5 O8 (Mn5O8)
 Gibbs Energy (900 K) = -18.7753,
 GibbsComputedEntry | mp-18759 | Mn6 O8 (Mn3O4)
 Gibbs Energy (900 K) = -22.0535,
 GibbsComputedEntry | mp-35 | Mn29 (Mn)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
 Gibbs Energy (900 K) = -121.0591,
 GibbsComputedEntry | mp-22508 | Y1 Mn12 (YMn12)
 Gibbs Energy (900 K) = -0.0997,
 GibbsComputedEntry | mp-1187739 | Y3 (Y)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-18831 | Y4 Mn4 O14 (Y2Mn2O7)
 Gibbs Energy (900 K) = -52.1371,
 GibbsComputedEntry | mp-510598 | Y4 Mn8 O20 (YMn2O5)
 Gibbs Energy (900 K) = -68.6480,
 GibbsComputedEntry | mp-19385 | Y6 Mn6 O18 (YMnO3)
 Gibbs Energy (900 K) = -76.9563,
 GibbsComputedEntry | m

## Running enumerators

There are several types of enumerator classes contained within `rxn_network.enumerators`: These are:

1. `BasicEnumerator`: use a combinatorial approach to identify all possible (closed) reactions within a set of entries
2. `BasicOpenEnumerator`: use a combinatorial approach to identify all **open** reactions within a set of entries and a list of specified open entries/elements
3. `MinimizeGibbsEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing Gibbs free energy between a set of two reacting phases touching at an interface
4. `MinimizeGrandPotentialEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing the grand potential between a set of two reacting phases touching at an interface with an **open** element at a specified chemical potential

### Basic enumerators
We first create a basic enumerator object by initializing one from the `BasicEnumerator` class:

In [7]:
be = BasicEnumerator()

The `BasicEnumerator` class, as is true for all other enumerator classes, can be provided with several arguments for customizing the enumerator output. An explanation of some of these is given below:

- **precursors**: Optional list of precursor formulas; only reactions
    which contain at least these phases as reactants will be enumerated. See
    the "exclusive_precursors" parameter for more details.
- **targets**: Optional list of target formulas; only reactions which include
    formation of at least one of these targets will be enumerated. See the
    "exclusive_targets" parameter for more details.
- **calculators**: Optional list of Calculator object names to be initialized; see
    calculators module for options (e.g., ["ChempotDistanceCalculator"])
- **n**: Maximum reactant/product cardinality; i.e., largest possible number of
    entries on either side of the reaction. Defaults to 2.
- **exclusive_precursors**: Whether to consider only reactions that have reactants 
    which are a subset of the provided list of precursors. In other
    words, if True, this only identifies reactions with reactants selected
    from the precursors argument. Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the
    form products that are a subset of the provided list of targets. If
    False, this only identifies reactions with no unspecified byproducts.
    Defualts to False.
- **remove_unbalanced**: Whether to remove reactions which are unbalanced.
    Defaults to True.
- **remove_changed**: Whether to remove reactions which can only be balanced by
    removing a reactant/product or having it change sides. Defaults to True.
- **calculate_e_above_hulls**: Whether to calculate e_above_hull for each entry
    upon initialization of the entries at the beginning of enumeration.
- **quiet**: Whether to run in quiet mode (no progress bar). Defaults to False.

Note that the default arguments are good for generating a list of simple (unconstrained) reactions, as we might build for a reaction network. Run the following cell:

In [8]:
all_rxns = be.enumerate(filtered_entries)

2022-10-20 15:04:47,295	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.15s/it]


This should complete somewhat quickly (within a few seconds). As a result, a list of ~700 generated reactions will be stored within the `all_rxns` object:

In [9]:
print(len(all_rxns))

692


The reactions returned by the enumerators are placed in a memory-efficient class called `ReactionSet`. 

This class stores sets of reactions as arrays. The actual reaction objects  can only be accessed by iterating through the reaction set. Lets print the first 10 reactions:

In [10]:
for count, r in enumerate(all_rxns):
    print(r)
    if count>10:
        break # first 10 reactions (may be different on your workstation)

YMn12 -> Y + 12 Mn
Y + 12 Mn -> YMn12
0.5 Mn3O4 -> O2 + 1.5 Mn
O2 + 1.5 Mn -> 0.5 Mn3O4
2 Mn3O4 -> Mn + Mn5O8
Mn + Mn5O8 -> 2 Mn3O4
Mn3O4 -> Mn + 2 MnO2
Mn + 2 MnO2 -> Mn3O4
3 Mn3O4 -> Mn + 4 Mn2O3
Mn + 4 Mn2O3 -> 3 Mn3O4
2 Mn3O4 -> O2 + 6 MnO
O2 + 6 MnO -> 2 Mn3O4


Looking at the list of reactions above, we see that all reactions are stoichiometrically balanced. If we look at any particular reaction object, we find that the reaction energy and uncertainty are properties which can be easily accessed:

In [11]:
r = list(all_rxns.get_rxns())[0]
print(r)
print(r.energy_per_atom, "±", r.energy_uncertainty_per_atom, "eV/atom")

YMn12 -> Y + 12 Mn
0.007670494437353557 ± 0.06815401979488214 eV/atom


What if you want to enumerate all reactions from a known set of precursors? An inefficient way to do so might be to filter the previous reaction set by your phases of interest.

A more efficient solution has been provided. We can supply our precursors when we initialize the `BasicEnumerator` object. This will reduce the number of calculations required significantly.

In [12]:
be_precursors = BasicEnumerator(precursors=["Y2O3"])
y2o3_rxns_exclusive = be_precursors.enumerate(filtered_entries)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 124.01it/s]


In [13]:
for r in y2o3_rxns_exclusive:
    print(r)

0.5 Y2O3 -> Y + 0.75 O2


Note that by default, this only produces reactions which **exclusively** have the provided precursor(s). 

To include reactions that contain this precursor (and possibly others) set the `exclusive_precursors=False`:

In [14]:
be_precursors = BasicEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
y2o3_rxns = be_precursors.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.84it/s]


We now see a much larger list of reactions, all of which contain Y2O3 as a precursor (and often another phase such as a manganese oxide):

In [15]:
for r in y2o3_rxns:
    print(r)

0.5 Y2O3 -> Y + 0.75 O2
Mn2O3 + Y2O3 -> 2 YMnO3
Y2O3 + 2 MnO2 -> Y2Mn2O7
16 Mn + 0.6667 Y2O3 -> O2 + 1.333 YMn12
35 Mn + 1.333 Y2O3 -> Mn3O4 + 2.667 YMn12
12.94 Mn + 0.5 Y2O3 -> YMn12 + 0.1875 Mn5O8
30 Mn + 1.667 Y2O3 -> YMn2O5 + 2.333 YMn12
13 Mn + Y2O3 -> YMnO3 + YMn12
12.75 Mn + 0.875 Y2O3 -> YMn12 + 0.375 Y2Mn2O7
12.75 Mn + 0.5 Y2O3 -> YMn12 + 0.75 MnO2
9 Mn + 0.3333 Y2O3 -> MnO + 0.6667 YMn12
26 Mn + Y2O3 -> Mn2O3 + 2 YMn12
1.125 Mn + 0.5 Y2O3 -> Y + 0.375 Mn3O4
0.9375 Mn + 0.5 Y2O3 -> Y + 0.1875 Mn5O8
0.8571 Mn + 0.7143 Y2O3 -> Y + 0.4286 YMn2O5
Mn + Y2O3 -> Y + YMnO3
0.75 Mn + 0.875 Y2O3 -> Y + 0.375 Y2Mn2O7
0.75 Mn + 0.5 Y2O3 -> Y + 0.75 MnO2
1.5 Mn + 0.5 Y2O3 -> Y + 1.5 MnO
Mn + 0.5 Y2O3 -> Y + 0.5 Mn2O3
Mn3O4 + 0.5714 Y2O3 -> 1.143 YMn2O5 + 0.7143 Mn
Mn5O8 + 1.143 Y2O3 -> 2.286 YMn2O5 + 0.4286 Mn
MnO + 0.1429 Y2O3 -> 0.2857 YMn2O5 + 0.4286 Mn
Mn2O3 + 0.4286 Y2O3 -> 0.8571 YMn2O5 + 0.2857 Mn
Mn3O4 + 1.333 Y2O3 -> 2.667 YMnO3 + 0.3333 Mn
MnO + 0.3333 Y2O3 -> 0.6667 YMnO3 + 0.33

This same approach can be used for the target phase(s) as well.

With `exclusive_targets=True`:

In [16]:
be_target = BasicEnumerator(targets=["YMnO3"], exclusive_targets=True)
ymno3_rxns = be_target.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 38.79it/s]


In [17]:
for r in ymno3_rxns:
    print(r)

Mn2O3 + Y2O3 -> 2 YMnO3


With `exclusive_targets=False` (the default):

In [18]:
be_target = BasicEnumerator(targets=["YMnO3"])
ymno3_rxns = be_target.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.52it/s]


In [19]:
for r in ymno3_rxns:
    print(r)

YMn2O5 -> MnO2 + YMnO3
Mn2O3 + Y2O3 -> 2 YMnO3
2 Y2Mn2O7 -> O2 + 4 YMnO3
13 Mn + Y2O3 -> YMnO3 + YMn12
Mn + Y2O3 -> Y + YMnO3
0.5 Mn + YMn2O5 -> YMnO3 + 0.5 Mn3O4
0.25 Mn + YMn2O5 -> YMnO3 + 0.25 Mn5O8
Mn + YMn2O5 -> YMnO3 + 2 MnO
0.5 Mn + 1.5 YMn2O5 -> Mn2O3 + 1.5 YMnO3
O2 + 0.6667 YMn12 -> 7.333 Mn + 0.6667 YMnO3
Mn3O4 + 1.333 YMn12 -> 17.67 Mn + 1.333 YMnO3
Mn5O8 + 2.667 YMn12 -> 34.33 Mn + 2.667 YMnO3
YMn2O5 + 0.6667 YMn12 -> 8.333 Mn + 1.667 YMnO3
Y2Mn2O7 + 0.3333 YMn12 -> 3.667 Mn + 2.333 YMnO3
MnO2 + 0.6667 YMn12 -> 8.333 Mn + 0.6667 YMnO3
MnO + 0.3333 YMn12 -> 4.667 Mn + 0.3333 YMnO3
Mn2O3 + YMn12 -> 13 Mn + YMnO3
Y + 0.75 Mn3O4 -> 1.25 Mn + YMnO3
Y2O3 + 0.75 Mn3O4 -> 0.25 Mn + 2 YMnO3
Y + 0.375 Mn5O8 -> 0.875 Mn + YMnO3
Y + 1.5 YMn2O5 -> 0.5 Mn + 2.5 YMnO3
Y + 1.5 MnO2 -> 0.5 Mn + YMnO3
Y + 3 MnO -> 2 Mn + YMnO3
Mn2O3 + Y -> Mn + YMnO3
Y2O3 + 3 MnO -> Mn + 2 YMnO3
0.375 Mn + 0.5 Y2Mn2O7 -> YMnO3 + 0.125 Mn3O4
0.3125 Mn + 0.5 Y2Mn2O7 -> YMnO3 + 0.0625 Mn5O8
0.3333 Mn + 0.6667 Y

And finally, with multiple targets specified (e.g., YMnO3 and O2):

In [20]:
be_targets = BasicEnumerator(targets=["YMnO3", "O2"], exclusive_targets=True)
ymno3_rxns_o2 = be_targets.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 21.35it/s]


In [21]:
for r in ymno3_rxns_o2:
    print(r)

Mn2O3 + Y2O3 -> 2 YMnO3
0.5 Y2Mn2O7 -> YMnO3 + 0.25 O2
Mn5O8 + 2.5 Y2O3 -> 5 YMnO3 + 0.25 O2
YMn2O5 + 0.5 Y2O3 -> 2 YMnO3 + 0.25 O2
Y2O3 + 2 MnO2 -> 2 YMnO3 + 0.5 O2


#### Open entries

In the previous cell, we showed that it was possible to specify YMnO$_3$ as a target, along with O$_2$. However, because O$_2$ is a gas, it is often desirable to include it as an **open entry** in addition to the 1-2 possible precursors/targets. For example, we may want to specify a reaction as follows:

$$ A + B ~ (+~O_2) \rightarrow C + D ~ (+~O_2) $$

To do this, we use the `BasicOpenEnumerator` class, which is an extension to the previous basic enumerator. All of the lessons learned above also apply to this class, although now a list of open entry formulas (e.g., `["O2"]`) must be specified.

In [22]:
be_target_open = BasicOpenEnumerator(open_phases=["O2"], targets=["YMnO3"])
ymno3_rxns_open = be_target_open.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.52it/s]


You will now see that the reactions from before are included in this new list, along with many other open-O$_2$ reactions -- some which even have a total of 3 reactants or 3 products. In other words, the open entry/entries do not count towards the specified $n$ of the reactions.

In [23]:
for r in ymno3_rxns_open:
    print(r)

YMn2O5 -> O2 + YMnO3 + Mn
11 YMn2O5 -> YMn12 + 10 YMnO3 + 12.5 O2
3 YMn2O5 -> O2 + 3 YMnO3 + Mn3O4
5 YMn2O5 -> O2 + 5 YMnO3 + Mn5O8
2 YMn2O5 -> O2 + 2 YMnO3 + 2 MnO
4 YMn2O5 -> O2 + 2 Mn2O3 + 4 YMnO3
Y + 1.5 O2 + Mn -> YMnO3
Y2O3 + 1.5 O2 + 2 Mn -> 2 YMnO3
YMn12 + 11 Y + 18 O2 -> 12 YMnO3
YMn12 + 5.5 Y2O3 + 9.75 O2 -> 12 YMnO3
Y + 0.8333 O2 + 0.3333 Mn3O4 -> YMnO3
Y2O3 + 0.1667 O2 + 0.6667 Mn3O4 -> 2 YMnO3
Y + 0.7 O2 + 0.2 Mn5O8 -> YMnO3
Y + 0.5 O2 + YMn2O5 -> 2 YMnO3
O2 + 2 Y + 2 MnO2 -> 2 YMnO3
Y + O2 + MnO -> YMnO3
Mn2O3 + 2 Y + 1.5 O2 -> 2 YMnO3
Y2O3 + 0.5 O2 + 2 MnO -> 2 YMnO3
2 Y2Mn2O7 -> O2 + 4 YMnO3
YMn12 + 1.5 O2 -> YMnO3 + 11 Mn
YMn12 + 8.833 O2 -> YMnO3 + 3.667 Mn3O4
10 Y2O3 + 4 Mn5O8 -> O2 + 20 YMnO3
YMn12 + 10.3 O2 -> YMnO3 + 2.2 Mn5O8
2 Y2O3 + 4 YMn2O5 -> O2 + 8 YMnO3
2 Y2O3 + 4 MnO2 -> O2 + 4 YMnO3
O2 + 0.08 YMn12 -> 0.88 MnO2 + 0.08 YMnO3
O2 + 0.1429 YMn12 -> 0.1429 YMnO3 + 1.571 MnO
O2 + 0.1026 YMn12 -> 0.5641 Mn2O3 + 0.1026 YMnO3


It is even possible to specify multiple open entries, allowing for even more complex reactions (such as ones with 4+ reactants or products). For example, specifiyng Y2O3 and O2 as both being open:

In [24]:
be_target_open2 = BasicOpenEnumerator(open_phases=["Y2O3", "O2"],targets=["YMnO3"])
ymno3_rxns_open2 = be_target_open2.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:05<00:00,  1.28it/s]


In [25]:
for i, r in enumerate(ymno3_rxns_open2):
    if i==10:  # print only first 10 reactions for brevity
        break 
        
    print(r)

YMn2O5 -> O2 + YMnO3 + Mn
11 YMn2O5 -> YMn12 + 10 YMnO3 + 12.5 O2
3 YMn2O5 -> O2 + 3 YMnO3 + Mn3O4
5 YMn2O5 -> O2 + 5 YMnO3 + Mn5O8
2 YMn2O5 -> O2 + 2 YMnO3 + 2 MnO
4 YMn2O5 -> O2 + 2 Mn2O3 + 4 YMnO3
Y2O3 + 1.5 O2 + 2 Mn -> 2 YMnO3
YMn12 + 5.5 Y2O3 + 9.75 O2 -> 12 YMnO3
Y2O3 + 0.1667 O2 + 0.6667 Mn3O4 -> 2 YMnO3
Y2O3 + 0.5 O2 + 2 MnO -> 2 YMnO3


This may be useful for modeling systems with 2 or more gaseous, liquid, or molten phases, such as $O_2$ and a molten salt (e.g. $LiCl$)

### Minimize Enumerators

The "minimize" enumerators produce reactions via a thermodynamic free energy minimization approach, rather than a purely combinatorial one. This means that reactions are produced directly from the compositional phase diagram, where a new convex hull is drawn connecting two compositions within a closed (Gibbs) or open (Grand Potential) system. See the `InterfacialReactivity` class within the _pymatgen_ package for more information.

**It is important to note that reactions produced with the minimize enumerators may overlap some with the basic enumerators, but the minimize enumerators have the restriction that all reactions they originally produce must have a negative reaction energy and result in a set of product phases which are stable with respect to each other (i.e. they share a facet of the phase diagram).** That being said, these enumerators (when unrestricted) will also supply the reverse (i.e. positive energy) reaction as well.

The `MinimizeGibbsEnumerator` contains similar arguments as the basic enumerators. See the docstrings for updated information:
- **precursors**: Optional formulas of precursors.
- **targets**: Optional formulas of targets; only reactions which make these targets will be enumerated.
- **calculators**: Optional list of Calculator object names; see calculators module for options (e.g., ["ChempotDistanceCalculator"])
- **exclusive_precursors**: Whether to consider only reactions that have reactants which are a subset of the provided list of precursors. Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the provided target directly (i.e. with no byproducts). Defualts to False.
- **quiet**: Whether to run in quiet mode (no progress bar). Defaults to False.

In [26]:
mge = MinimizeGibbsEnumerator()

The default arguments, as before, help produce all reactions in a set of entries, given minimal constraints.

In [27]:
rxns = mge.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.73it/s]


In [28]:
for i, r in enumerate(rxns):
    if i==10:  # print only first 10 reactions for brevity
        break 
        
    print(r)

12 Mn + Y -> YMn12
0.5 Mn5O8 -> Mn2O3 + 0.5 MnO2
Mn + 0.5 O2 -> MnO
3 Mn + 2 O2 -> Mn3O4
2 Mn + 1.5 O2 -> Mn2O3
Mn + O2 -> MnO2
0.25 Mn3O4 + 0.25 Mn -> MnO
0.125 Mn5O8 + 0.375 Mn -> MnO
0.5 Mn5O8 + 0.5 Mn -> Mn3O4
0.375 Mn5O8 + 0.125 Mn -> Mn2O3


And as before, we can specify various combinations of precursors and targets, as well as whether or not they should be "exclusive".

In [29]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
rxns = mge_precursors.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 34.42it/s]


In [30]:
for r in rxns:
    print(r)

0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO
0.6667 Mn5O8 + 0.3333 Y2O3 -> Mn2O3 + 0.6667 YMn2O5
0.625 Mn5O8 + 0.5 Y2O3 -> YMn2O5 + 0.375 Mn3O4
0.3333 Mn5O8 + 0.6667 Y2O3 -> YMnO3 + 0.3333 YMn2O5
2 MnO2 + Y2O3 -> Y2Mn2O7
2 MnO2 + 0.5 Y2O3 -> YMn2O5 + 0.25 O2
0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3
2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4


In [31]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3", "Mn2O3"])
rxns = mge_precursors.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 79.36it/s]


In [32]:
for r in rxns:
    print(r)

2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4
0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3


In [33]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=True)
rxns = mge_targets.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.45it/s]


In [34]:
for r in rxns:
    print(r)

0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3


In [35]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=False)
rxns = mge_targets.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.37it/s]


In [36]:
for r in rxns:
    print(r)

Mn + YMn2O5 -> YMnO3 + 2 MnO
0.5 Mn + YMn2O5 -> YMnO3 + 0.5 Mn3O4
0.5 Mn + 0.5 Y2Mn2O7 -> YMnO3 + 0.5 MnO
0.375 Mn + 0.5 Y2Mn2O7 -> YMnO3 + 0.125 Mn3O4
0.3333 Mn + 0.6667 Y2Mn2O7 -> YMnO3 + 0.3333 YMn2O5
YMn12 + 8.833 O2 -> YMnO3 + 3.667 Mn3O4
YMn12 + 7 O2 -> YMnO3 + 11 MnO
YMn12 + 14 Mn3O4 -> YMnO3 + 53 MnO
YMn12 + 13.25 Mn5O8 -> YMnO3 + 25.75 Mn3O4
YMn12 + 4.667 Mn5O8 -> YMnO3 + 34.33 MnO
0.03636 YMn12 + 0.9636 YMn2O5 -> YMnO3 + 0.4545 Mn3O4
0.06667 YMn12 + 0.9333 YMn2O5 -> YMnO3 + 1.667 MnO
0.025 YMn12 + 0.625 Y2Mn2O7 -> YMnO3 + 0.275 YMn2O5
0.02752 YMn12 + 0.4862 Y2Mn2O7 -> YMnO3 + 0.1009 Mn3O4
0.03448 YMn12 + 0.4828 Y2Mn2O7 -> YMnO3 + 0.3793 MnO
YMn12 + 26.5 MnO2 -> YMnO3 + 12.5 Mn3O4
YMn12 + 14 MnO2 -> YMnO3 + 25 MnO
YMn12 + 53 Mn2O3 -> YMnO3 + 39 Mn3O4
YMn12 + 14 Mn2O3 -> YMnO3 + 39 MnO
2 Mn3O4 + Y -> YMnO3 + 5 MnO
0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO
Mn3O4 + 2 Y2Mn2O7 -> YMnO3 + 3 YMn2O5
0.6667 Mn5O8 + Y -> YMnO3 + 2.333 MnO
1.25 Mn5O8 + Y -> YMnO3 + 1.75 Mn3O4
0.3333 Mn5O8 

#### Open entries

And once again, as before, we can do all the same analysis with open entries. This time, the grand potential is used as the thermodynamic free energy which is minimized:

$$ \Phi = G - \mu_iN_i $$

Where $i$ is the open species with chemical potential $\mu_i$ with a molar amount $N_i$.

In [37]:
mgpe = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0)
open_rxns = mgpe.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.32it/s]


In [38]:
for r in open_rxns:
    print(r)

Mn + O2 -> MnO2
0.3333 Mn3O4 + 0.3333 O2 -> MnO2
0.2 Mn5O8 + 0.2 O2 -> MnO2
MnO + YMnO3 + 0.5 O2 -> YMn2O5
MnO + 0.5 O2 -> MnO2
0.5 Mn2O3 + 0.25 O2 -> MnO2
0.5 Mn2O3 + YMnO3 + 0.25 O2 -> YMn2O5
4 MnO2 + 2 Y2Mn2O7 -> O2 + 4 YMn2O5
MnO + 0.5 Y2Mn2O7 + 0.25 O2 -> YMn2O5
YMn12 + 12.5 O2 -> YMn2O5 + 10 MnO2
2 YMnO3 + 0.5 O2 -> Y2Mn2O7
2 Mn + Y + 2.5 O2 -> YMn2O5
2 Mn + 2 Y + 3.5 O2 -> Y2Mn2O7
2 Y + 1.5 O2 -> Y2O3
2 Mn + 0.5 Y2O3 + 1.75 O2 -> YMn2O5
2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7
Mn + YMnO3 + O2 -> YMn2O5
Mn + 0.5 Y2Mn2O7 + 0.75 O2 -> YMn2O5
0.1667 YMn12 + 0.8333 Y + 2.5 O2 -> YMn2O5
0.1667 YMn12 + 1.833 Y + 3.5 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.4167 Y2O3 + 1.875 O2 -> YMn2O5
0.09091 YMn12 + 0.9091 YMnO3 + 1.136 O2 -> YMn2O5
0.09091 YMn12 + 0.4545 Y2Mn2O7 + 0.9091 O2 -> YMn2O5
0.6667 Mn3O4 + Y + 1.167 O2 -> YMn2O5
0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7
0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7
0.6667 Mn3O4 + 0.5 Y2O3 + 0.4167 O2 -> YMn2O5
0.333

Note that the reaction objects returned are **NOT** (by default) configured to report their energy as a change in the grand potential.

To configure this, we need to transform these reactions to `OpenComputedReaction` objects. This class allows for easy specification of reactions where one of the elements is assigned a chemical potential.

This can be easily done by creating a new `ReactionSet` from the old one and specifying an open element and chemical potential during creation:

In [60]:
new_open_rxns = ReactionSet.from_rxns(open_rxns, open_elem="O", chempot=0.0)

We should now see that `(mu_O=0.0)` appears in the repr of the object:

In [40]:
sample_open_rxn = list(new_open_rxns)[0]
print(sample_open_rxn.__class__.__name__)

sample_open_rxn

OpenComputedReaction


Mn + O2 -> MnO2 (mu_O=0.0)

We can also, as before, customize which precursors and targets are specified:

In [41]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   precursors=["Y2O3"], exclusive_precursors=False)
open_rxns_precursors = mgpe_precursors.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 20.38it/s]


In [42]:
for r in open_rxns_precursors:
    print(r)

2 Mn + 0.5 Y2O3 + 1.75 O2 -> YMn2O5
2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.4167 Y2O3 + 1.875 O2 -> YMn2O5
0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7
0.6667 Mn3O4 + 0.5 Y2O3 + 0.4167 O2 -> YMn2O5
0.4 Mn5O8 + 0.5 Y2O3 + 0.15 O2 -> YMn2O5
0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7
0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7
8 MnO2 + 2 Y2O3 -> O2 + 4 YMn2O5
2 MnO + Y2O3 + O2 -> Y2Mn2O7
2 MnO + 0.5 Y2O3 + 0.75 O2 -> YMn2O5
Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7
Mn2O3 + 0.5 Y2O3 + 0.25 O2 -> YMn2O5


In [43]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.15it/s]


In [44]:
for r in open_rxns_targets:
    print(r)

2 YMnO3 + 0.5 O2 -> Y2Mn2O7
2 Mn + 2 Y + 3.5 O2 -> Y2Mn2O7
2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7
0.1667 YMn12 + 1.833 Y + 3.5 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7
0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7
0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7
0.4 Mn5O8 + 2 Y + 1.9 O2 -> Y2Mn2O7
0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7
Y + YMn2O5 + O2 -> Y2Mn2O7
2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7
2 MnO + 2 Y + 2.5 O2 -> Y2Mn2O7
Mn2O3 + 2 Y + 2 O2 -> Y2Mn2O7
0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7
2 MnO + Y2O3 + O2 -> Y2Mn2O7
Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7


Note that setting the chemical potential to a value outside of the range of stability of the target (e.g., `mu_O = -3`, causes the enumerator to yield no reactions:

In [45]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=-3,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.51it/s]


In [46]:
print(len(open_rxns_targets))

0


It is encouraged for you to try out what you have learned above on your own system of interest. Please feel free to reach out if you have any questions. You can either email the package maintainer or raise an Issue here: https://github.com/GENESIS-EFRC/reaction-network/issues

## Running enumerators with the _jobflow_ package

The _jobflow_ package is used to develop and run computational workflows. The _reaction-network_ package has several jobs (and flows) written using jobflow.

The `ReactionEnumerationMaker` (see below) has been created to run a list of enumerators on a provided entry set. This job can be used either by itself or as part of a larger flow. The latter will be showed in the next example notebook on reaction network creation.

In [47]:
from jobflow.managers.local import run_locally
from rxn_network.jobs.core import ReactionEnumerationMaker

In [56]:
maker = ReactionEnumerationMaker()
job = maker.make([BasicEnumerator()], filtered_entries)

The job can now be run either locally (as shown here) or launched on a remote workstation using the _fireworks_ package. Please see the jobflow documentation for more info on how to do this: https://materialsproject.github.io/jobflow/.

The following cell will only work if you have configured your jobflow settings correctly (which means providing connection information for a database so that jobflow knows where to store its outputs!)

In [57]:
output = run_locally(job)

2022-10-20 15:08:12,827 INFO Started executing jobs locally
2022-10-20 15:08:12,831 INFO Starting job - enumerate_reactions (604d00cc-066c-4cb0-a3a3-3c9d6cd6334b)
2022-10-20 15:08:12,835 INFO rxn_network.jobs.core Running enumerators...
2022-10-20 15:08:12,836 INFO rxn_network.jobs.utils Running BasicEnumerator


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.06it/s]

2022-10-20 15:08:14,164 INFO rxn_network.jobs.utils Adding 692 reactions to reaction set
2022-10-20 15:08:14,164 INFO rxn_network.jobs.utils Completed reaction enumeration. Filtering duplicates...
2022-10-20 15:08:14,170 INFO rxn_network.jobs.core Building task document...


2022-10-20 15:08:14,547 INFO Finished job - enumerate_reactions (604d00cc-066c-4cb0-a3a3-3c9d6cd6334b)
2022-10-20 15:08:14,548 INFO Finished executing jobs locally


The final reaction set can be accessed from the output dictionary like such:

In [58]:
output[job.uuid][1].output.rxns

### Thank you!

We hope this notebook was helpful in introducing the enumerator classes found in the _reaction-network_ package. If any significant errors are encountered, please first double-check that your settings are configured properly (e.g., proper installation of all dependencies, acquisition of a Materials Project API key, etc.). 

If the error persists, then please raise an Issue here: https://github.com/GENESIS-EFRC/reaction-network/issues